In [ ]:
import logging
import os
import sys
from collections import defaultdict
from pathlib import Path
from typing import Dict, List, Literal, Optional, Tuple, Union

import IPython
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from matplotlib.axes import Axes as Axes

notebook_path = Path(IPython.extract_module_locals()[1]["__vsc_ipynb_file__"])
project_dir = notebook_path.parent.parent
sys.path.append(str(project_dir))
import src.utils.custom_log as custom_log
from src.utils.CfCFilter import CfCFilter
from src.utils.Csv import Csv
from src.utils.iso18571 import rating_iso_18571
from src.utils.ReadBinout import ReadBinout
from src.utils.set_rcparams import set_rcparams
from src.utils.UnifySignal import UnifySignal

os.chdir(project_dir)
LOG = logging.getLogger(__name__)
custom_log.init_logger(logging.INFO)
LOG.info("Log initialized")

set_rcparams()

In [ ]:
B_PATH: Path = Path("/mnt") / "q" / "Val_Chain_Sims" / "AB_Testing" / "PAB_Reduced_Env" / "Steps"
LOG.info("Base path: %s %s", B_PATH, B_PATH.is_dir())
VERSIONS: List[str] = [
    "Stand_PAB_Orig",
    "Stand_PAB_Simple",
    # "Stand_PAB_Simple_Delay",
]
VERSION_NAMES: Dict[str, str] = {
    VERSIONS[0]: "Original PAB in Stand Test",
    VERSIONS[1]: "Simplified PAB in Stand Test",
    # VERSIONS[2]: "Simplified Delayed",
}

### Get Data

In [ ]:
def get_data() -> pd.DataFrame:
    unifier = UnifySignal(target_sampling_rate_ms=0.1, target_tend_ms=130, target_tstart_ms=15)
    fields = {
        "abstat": (
            "pressure",
            "volume",
            "total_mass",
            "gas_temp",
            "dm_dt_in",
            "dm_dt_out",
        ),
        "glstat": (
            "total_energy",
            "added_mass",
            "kinetic_energy",
            "internal_energy",
            "sliding_interface_energy",
            "time_step",
            "hourglass_energy",
        ),
    }
    dbs = []
    for version in VERSIONS:
        LOG.info(f"Reading {version}")
        with ReadBinout(sim_dir=B_PATH / version) as binout:
            db_version = []
            for field, channels in fields.items():
                db = pd.DataFrame(index=binout.binout.read(field, "time") * 1000)
                for key in channels:
                    db[key] = binout.binout.read(field, key)
                db = unifier.unify(db=db)
                db_version.append(db)
            db_version = pd.concat(db_version, axis=1)
            db_version["version"] = [VERSION_NAMES[version]] * db_version.shape[0]
            dbs.append(db_version.copy())

    dbs = pd.concat(dbs, axis=0)
    dbs["Time [ms]"] = dbs.index
    return dbs


DB = get_data()
DB

In [ ]:
def get_isos() -> pd.DataFrame:
    isos = defaultdict(dict)
    for channel in ("dm_dt_in", "dm_dt_out", "gas_temp", "pressure", "total_mass", "volume"):
        for version in DB["version"].unique():
            if version != "Original":
                LOG.info(f"Calculating ISO for {channel} in {version}")
                isos[channel][version] = rating_iso_18571(
                    signal_ref=DB[channel][DB["version"].eq("Original")].to_numpy(),
                    signal_comp=DB[channel][DB["version"].eq(version)].to_numpy(),
                )["ISO 18571 Rating"]

    return pd.DataFrame(isos)


ISOS = get_isos()
ISOS

In [ ]:
for ch in DB.columns:
    if ch not in {"Time [ms]", "version"}:
        fig, ax = plt.subplots()
        sns.lineplot(data=DB, x="Time [ms]", y=ch, hue="version", ax=ax)
        ax.set_title(ch)
        ax.axvline(60, color="black", linestyle="--")

In [ ]:
def plot():
    fig_width: float = 1 * (448.13095 / 72) -0.2
    fig_height = 0.3 * fig_width
    chs = ["volume", "pressure", "dm_dt_in", "dm_dt_out"]
    fig, ax = plt.subplot_mosaic(
        mosaic=[["L"]*4, chs],
        sharex=True,
        layout="constrained",
        height_ratios=[0.1,1],
    )
    db = DB.copy()
    for ch in chs:
        if ch == "volume":
            db[ch] *= 1e-6
        elif "dm" in ch:
            db[ch] *= 1e3
        g = sns.lineplot(data=db, x="Time [ms]", y=ch, hue="version", ax=ax[ch], palette=["orchid", "cadetblue"])
        ax[ch].legend().remove()
        ax[ch].axvline(60, color="black", linestyle="--", lw=1)
        ax[ch].grid()

        ax[ch].set_ylabel(
            {
                "volume": "Volume [l]",
                "pressure": "Absolute Pressure [MPa]",
                "dm_dt_in": "Mass Flow In [g/ms]",
                "dm_dt_out": "Mass Flow Out [g/ms]",
            }[ch]
        )
        ax[ch].set_xticks(np.arange(20, 121, 20))

    ax["L"].axis("off")
    ax["L"].legend(*ax[ch].get_legend_handles_labels(), loc="upper center", ncol=2)
    fig.set_figheight(fig_height)
    fig.set_figwidth(fig_width)
    fig.savefig(Path("reports") / "pab_stand_test_comp.pdf")


plot()